1. Какую из этих гипотез нельзя проверить с помощью критерия знаков?

    Гипотеза о равенстве средних в двух независимых выборках
    

2. Какую из этих гипотез о средних двух независимых выборок проверяет критерий Манна-Уитни?
    
    F1(x) = F2(x) против F1(x) = F2(x + delta)


3. У вас есть две регрессионные модели и двадцать наборов данных. На обучающей подвыборке каждого набора вы 
   настраиваете обе модели и считаете среднеквадратичную ошибку на тестовой подвыборке набора. Какими из этих критериев
   не стоит проверять гипотезу о том, что среднее качество регрессионных моделей одинаково?

    Манна-Уитни, Стьюдент для независимых

In [53]:
import numpy as np
import pandas as pd
import itertools

from scipy import stats
from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import zconfint

# 4

X = np.array([49,58,75,110,112,132,151,276,281,362])
round(stats.wilcoxon(X - 200)[1], 4)

0.2845

In [46]:
# 5

X1 = [22,22,15,13,19,19,18,20,21,13,13,15]
X2 = [17,18,18,15,12,4,14,15,10]

round(stats.mannwhitneyu(X1, X2, alternative='greater')[1], 4)

0.029

In [10]:
# 6

X = pd.read_csv('challenger.txt', sep='\t')

In [13]:
X1 = X[X['Incident'] == 0]['Temperature'].values
X2 = X[X['Incident'] == 1]['Temperature'].values

In [15]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def confint(stat, alpha):
    return (np.percentile(stat, [alpha / 2 * 100, (1 - alpha / 2) * 100]))

In [43]:
np.random.seed(0)

p1 = get_bootstrap_samples(X1, 1000)
p2 = get_bootstrap_samples(X2, 1000)

stat = np.mean(p1, axis = 1) - np.mean(p2, axis = 1)
round(confint(stat, 0.05)[0], 4)

1.423

In [35]:
X1, X2

(array([18.9, 20.6, 20. , 19.4, 22.2, 22.8, 21.1, 25.6, 19.4, 19.4, 23.9,
        21.1, 27.2, 24.4, 26.1, 24.4]),
 array([21.1, 13.9, 17.2, 21.1, 11.7, 23.9, 14.4]))

In [36]:
# 7

def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [37]:
def permutation_test(sample1, sample2, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample1, sample2)
    
    zero_distr = permutation_zero_dist_ind(sample1, sample2, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [38]:
np.random.seed(0)
permutation_test(X1, X2, max_permutations = 10000)

0.007